In [30]:
class System:
  def __init__(self):
    self.facts = {}
    self.rules = []
    self.inferred_facts = {}
    self.goal = None

  def print_all_facts(self):
    print("\nAll Known Facts:")
    print("Initial Facts:")
    for attr, value in sorted(self.facts.items()):
      print(f"  {attr}: {value}")

    print("Inferred Facts:")
    for attr, value in sorted(self.inferred_facts.items()):
      print(f"  {attr}: {value}")

  def read_facts(self, filename):
    with open(filename, 'r') as file:
      content = file.read()

    lines = content.strip().split('\n')
    for line in lines:
      line = line.strip()
      if not line:
        continue
      if line.startswith("#goal prove"):
        goal_str = line[12:].strip()
        if " is " in goal_str:
          attr, value = goal_str.split(" is ", 1)
          self.goal = {"type": "is", "attribute": attr.strip(), "value": value.strip()}
        else:
          self.goal = {"type": "bool", "attribute": goal_str}
        continue
      if 'is' in line:
        attribute, value = line.split('is')
        self.facts[attribute.strip()] = value.strip()
      elif '=' in line:
        attribute, value = line.split('=')
        try:
          self.facts[attribute.strip()] = float(value.strip())
        except:
          self.facts[attribute.strip()] = value.strip()
      else:
        self.facts[line] = True
    print(f"Parsed {len(self.facts)} initial facts")

  def read_rules(self, filename):
    with open(filename, 'r') as file:
      content = file.read()

    lines = content.split('\n')
    for line in lines:
      if line.startswith("IF"):
        parts = line[3:].split(" THEN ")
        if len(parts) == 2:
          condition_str = parts[0]
          conclusion_str = parts[1]
          conditions = []
          if " AND " in condition_str:
            condition_list = condition_str.split(" AND ")
            for cond in condition_list:
              conditions.append(self._parse_condition(cond))
          else:
            conditions.append(self._parse_condition(condition_str))
          conclusion = self._parse_condition(conclusion_str)
          self.rules.append({
            "conditions": conditions,
            "conclusion": conclusion
          })
    print(f"Parsed {len(self.rules)} rules")

  def _parse_condition(self, condition_str):
    if " OR " in condition_str:
      or_parts = condition_str.split(" OR ")
      return {"type": "OR", "parts": [self._parse_simple_condition(part) for part in or_parts]}

    return self._parse_simple_condition(condition_str)

  def _parse_simple_condition(self, condition_str):
    if 'is' in condition_str:
      attribute, value = condition_str.split('is')
      return {"type": "is", "attribute": attribute.strip(), "value": value.strip()}
    elif '<' in condition_str:
      attribute, value = condition_str.split('<', 1)
      return {"type": "<", "attribute": attribute.strip(), "value": float(value.strip())}
    elif '>' in condition_str:
      attribute, value = condition_str.split('>')
      return {"type": ">", "attribute": attribute.strip(), "value": float(value.strip())}
    else:
      return {"type": "bool", "attribute": condition_str.strip()}

  def evaluate(self, condition, combined_facts):
    if condition["type"] == "is":
      return (condition["attribute"] in combined_facts and
              str(combined_facts[condition["attribute"]]) == str(condition["value"]))
    if condition["type"] == "<":
      return (condition["attribute"] in combined_facts and
              isinstance(combined_facts[condition["attribute"]], (int, float)) and
              combined_facts[condition["attribute"]] < condition["value"])
    if condition["type"] == ">":
      return (condition["attribute"] in combined_facts and
              isinstance(combined_facts[condition["attribute"]], (int, float)) and
              combined_facts[condition["attribute"]] > condition["value"])
    if condition["type"] == "bool":
      return condition["attribute"] in combined_facts and combined_facts[condition["attribute"]] is True
    if condition["type"] == "OR":
      for part in condition["parts"]:
        if self.evaluate(part, combined_facts):
          return True
      return False
    return False

  def format_condition(self, condition):
    if condition["type"] == "OR":
      return " OR ".join([self.format_condition(part) for part in condition["parts"]])
    elif condition["type"] == "is":
      return f"{condition['attribute']} is {condition['value']}"
    elif condition["type"] == "<":
      return f"{condition['attribute']} < {condition['value']}"
    elif condition["type"] == ">":
      return f"{condition['attribute']} > {condition['value']}"
    elif condition["type"] == "bool":
      return condition["attribute"]
    return "unknown condition"

  def format_rule(self, rule_idx):
    """Format a rule into a readable string."""
    rule = self.rules[rule_idx]
    conditions = " AND ".join([self.format_condition(cond) for cond in rule["conditions"]])
    conclusion = self.format_condition(rule["conclusion"])
    return f"Rule {rule_idx+1}: IF {conditions} THEN {conclusion}"

  def print_facts_state(self, iteration, all_facts):
    """Print the current state of all facts."""
    print(f"\nFacts after iteration {iteration}:")
    for attr, value in sorted(all_facts.items()):
      print(f"  {attr}: {value}")

  def add_fact(self, fact_key, fact_value):
    """Add a new fact to inferred facts."""
    self.inferred_facts[fact_key] = fact_value

  def conclusion_matches_goal(self, conclusion, goal):
    if goal["type"] == "OR":
      for part in goal["parts"]:
        if self.conclusion_matches_goal(conclusion, part):
          return True
      return False

    if conclusion["type"] != goal["type"]:
      return False

    if conclusion["type"] == "is":
      return (conclusion["attribute"] == goal["attribute"] and
              conclusion["value"] == goal["value"])
    elif conclusion["type"] in ["<", ">"]:
      return (conclusion["attribute"] == goal["attribute"] and
              conclusion["value"] == goal["value"])
    elif conclusion["type"] == "bool":
      return conclusion["attribute"] == goal["attribute"]

    return False

  def backward_chaining(self, goal=None):

    print("\n===== BACKWARD CHAINING =====")
    if goal is None:
      goal = self.goal

    if goal is None:
      print("No goal specified for backward chaining.")
      return False

    print(f"Goal to prove: {self.format_condition(goal)}")

    all_facts = {**self.facts, **self.inferred_facts}

    print("\nInitial facts for backward chaining:")
    for attr, value in sorted(all_facts.items()):
      print(f"  {attr}: {value}")

    proved_goals = set()
    result = self._backward_chain(goal, all_facts, proved_goals)

    if result:
      print(f"\nSuccessfully proved: {self.format_condition(goal)}")
    else:
      print(f"\nFailed to prove: {self.format_condition(goal)}")

    return result

  def _backward_chain(self, goal, facts_dict, proved_goals, depth=0):

    indent = "  " * depth
    goal_str = self.format_condition(goal)
    print(f"{indent}Step {depth + 1}: Trying to prove: {goal_str}")

    print(f"{indent}Current facts at depth {depth}:")
    for attr, value in sorted(facts_dict.items()):
      print(f"{indent}  {attr}: {value}")

    if self.evaluate(goal, facts_dict):
      print(f"{indent}Step {depth + 1}: Goal is directly satisfied by facts: {goal_str}")
      return True

    goal_key = str(goal)
    if goal_key in proved_goals:
      print(f"{indent}Step {depth + 1}: Goal was previously proven: {goal_str}")
      return True

    applicable_rules = []
    for rule_idx, rule in enumerate(self.rules):
      if self.conclusion_matches_goal(rule["conclusion"], goal):
        applicable_rules.append((rule_idx, rule))

    if not applicable_rules:
      print(f"{indent}Step {depth + 1}: No rules found that can prove: {goal_str}")
      return False

    for rule_idx, rule in applicable_rules:
      print(f"{indent}Step {depth + 1}: Considering {self.format_rule(rule_idx)}")

      all_conditions_proven = True
      for condition_idx, condition in enumerate(rule["conditions"]):
        condition_str = self.format_condition(condition)
        print(f"{indent}  Step {depth + 2}.{condition_idx + 1}: Trying to prove condition: {condition_str}")

        if not self._backward_chain(condition, facts_dict, proved_goals, depth + 2):
          all_conditions_proven = False
          print(f"{indent}  Step {depth + 2}.{condition_idx + 1}: Failed to prove condition: {condition_str}")
          break

      if all_conditions_proven:
        if goal["type"] == "is":
          facts_dict[goal["attribute"]] = goal["value"]
          print(f"{indent}Step {depth + 1}: Added new fact: {goal['attribute']} is {goal['value']}")
        elif goal["type"] == "bool":
          facts_dict[goal["attribute"]] = True
          print(f"{indent}Step {depth + 1}: Added new fact: {goal['attribute']}")

        print(f"{indent}Step {depth + 1}: Proved goal: {goal_str}")
        proved_goals.add(goal_key)

        print(f"{indent}Updated facts after proving {goal_str}:")
        for attr, value in sorted(facts_dict.items()):
          print(f"{indent}  {attr}: {value}")

        return True

    print(f"{indent}Step {depth + 1}: Failed to prove goal: {goal_str}")
    return False


  def forward_chaining(self):
        print("\n===== FORWARD CHAINING =====")
        self.inferred_facts = {}
        current_iteration = 0
        found_new = True

        full_known = dict(self.facts)
        self.print_facts_state(current_iteration, full_known)

        goal = self.goal
        print(f"Goal to prove: {self.format_condition(goal)}")

        while found_new:
            current_iteration += 1
            found_new = False
            print(f"\nIteration {current_iteration}...")
            if goal is not None:
                if self.evaluate(goal, self.facts):
                    print(f"Goal already satisfied by initial facts: {self.format_condition(goal)}")
                    break
                
            combined = {**self.facts, **self.inferred_facts}

            for idx, rule in enumerate(self.rules):
                if all(self.evaluate(cond, combined) for cond in rule["conditions"]):
                    concl = rule["conclusion"]
                    if concl["type"] == "is":
                        k, v = concl["attribute"], concl["value"]
                        if (k not in self.facts and
                            (k not in self.inferred_facts or str(self.inferred_facts[k]) != str(v))):
                            self.inferred_facts[k] = v
                            found_new = True
                            print(f"  Rule {idx+1} applied -> {k} is {v}")

                    elif concl["type"] == "bool":
                        k = concl["attribute"]
                        if k not in self.facts and k not in self.inferred_facts:
                            self.inferred_facts[k] = True
                            found_new = True
                            print(f"  Rule {idx+1} applied -> {k} is True")

            updated = {**self.facts, **self.inferred_facts}
            self.print_facts_state(current_iteration, updated)

        print("\nDone. Total iterations:", current_iteration)

In [33]:
# Example usage
def main():
    # Create and initialize the system
    system = System()

    # Parse rules and facts from files
    system.read_rules("rules.txt")
    system.read_facts("facts.txt")



    # If a goal was specified in the facts file, run backward chaining
    if system.goal:
        system.backward_chaining()


if __name__ == "__main__":
    main()


Parsed 13 rules
Parsed 5 initial facts

===== BACKWARD CHAINING =====
Goal to prove: citrus_fruit

Initial facts for backward chaining:
  citrus_fruit: True
  color: orange
  diameter: 7.0
  seeds: 0.0
  skin_smell: True
Step 1: Trying to prove: citrus_fruit
Current facts at depth 0:
  citrus_fruit: True
  color: orange
  diameter: 7.0
  seeds: 0.0
  skin_smell: True
Step 1: Goal is directly satisfied by facts: citrus_fruit

Successfully proved: citrus_fruit


In [32]:
def main():
    # Create and initialize the system
    system = System()

    # Parse rules and facts from files
    system.read_rules("rules.txt")
    system.read_facts("facts.txt")

    # Run forward chaining
    system.forward_chaining()

    # Print all facts after inference
    system.print_all_facts()

if __name__ == "__main__":
    main()

Parsed 13 rules
Parsed 5 initial facts

===== FORWARD CHAINING =====

Facts after iteration 0:
  citrus_fruit: True
  color: orange
  diameter: 7.0
  seeds: 0.0
  skin_smell: True
Goal to prove: citrus_fruit

Iteration 1...
Goal already satisfied by initial facts: citrus_fruit

Done. Total iterations: 1

All Known Facts:
Initial Facts:
  citrus_fruit: True
  color: orange
  diameter: 7.0
  seeds: 0.0
  skin_smell: True
Inferred Facts:
